In [ ]:
import re
from zipfile import ZipFile

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import scienceplots  # noqa
from tqdm import tqdm

from base_opt.utilities import eval_utils
from base_opt.utilities.file_locations import ROOT

# Set up LaTeX rendering
plt.rc('text', usetex=True)
plt.rc('text.latex', preamble=r"""
    \usepackage{siunitx}
""")

In [ ]:
import uuid

# csv dtype definition
csv_dtype = {
    'Step': int,
    # 'Action': 'numpy.ndarray',
    'Reward': float,
    'Solution': 'string',
    'Valid Solution': bool,
    'Run Time': float,
    'Fail Reason': 'string',
    'Reward Fail': float,
    'Optimizer Runtime': float,
    'Task ID': 'category',
    'Algorithm': 'category',
    'Optimizer Spec': 'string',
    'Seed': int,
    'Success': bool,
}

In [ ]:
# Extract raw data
ZipFile(ROOT.joinpath('data', 'outdir_test_1200.zip')).extractall(ROOT.joinpath('data', 'outdir_test_1200'))

In [ ]:
# Alternative file locator - load mixed GNU parallels output with Set/, Space/, Alg/, Seed/ directories
files = {}
for csv_file in tqdm(ROOT.joinpath('data', 'outdir_test_1200').rglob('**/*_raw.csv')):
    # Find Task Set, Action Space, Algorithm, Seed
    if re.search(r'(?<=Set/)(.*?)/', str(csv_file)) is None:
        if 'outdir_simple' in str(csv_file):
            task_set = 'test_simple'
        elif 'outdir_edge' in str(csv_file):
            task_set = 'test_edge'
    else:
        task_set = re.search(r'(?<=Set/)(.*?)/', str(csv_file)).group(0)[:-1]
    action = re.search(r'(?<=Space/)(.*?)/', str(csv_file)).group(0)[:-1]
    algorithm = re.search(r'(?<=Alg/)(.*?)/', str(csv_file)).group(0)[:-1]
    seed = re.search(r'(?<=Seed/)(\d)+_raw.csv', str(csv_file)).group(0)[:-8]  # Remove _raw.csv
    files[csv_file] = (task_set, action, algorithm, seed)

result_df = []
for file, (task_set, action, algorithm, seed) in tqdm(files.items()):
    df = pd.read_csv(file, dtype=csv_dtype)
    df['Task Set'] = task_set
    df['Action Space'] = action
    assert df['Algorithm'].nunique() == 1
    assert df['Seed'].nunique() == 1
    assert df['Algorithm'].unique()[0] == algorithm
    assert df['Seed'].unique()[0] == int(seed)
    result_df.append(df)
    
result_df = pd.concat(result_df, ignore_index=True, axis=0)

In [ ]:
from base_opt.base_opt.BaseOptimizer import BOOptimizer

timeout = 1200
n_initial_points = BOOptimizer.best_hps['n_initial_points']

In [ ]:
result_df.columns

In [ ]:
# Split edge case task set
result_df.loc[result_df['Task Set'] == 'test_edge', 'Task Set'] += "_" + result_df.loc[result_df['Task Set'] == 'test_edge', 'Task ID'].str.extract(r"(?<=base_opt\/edge_case\/).*((?:medium)|(?:hard))")[0]

In [ ]:
# Sanity checks
print(f"{result_df['Task Set'].unique() = }")
print(f"{result_df['Action Space'].unique() = }")
print(f"{result_df['Algorithm'].unique() = }")
print(f"{result_df['Seed'].unique() = }")

In [ ]:
result_df_clean = eval_utils.cleanup_preprocess_results(
    result_df,
    group_by=['Task Set', 'Action Space', 'Algorithm', 'Task ID', 'Seed'],)

In [ ]:
result_df_clean

In [ ]:
# Find best solution ID per task id, algorithm and action space
import pickle
best_solutions = result_df_clean[result_df_clean['Valid Solution']].sort_values('Reward', ascending=False).drop_duplicates(['Task ID', 'Algorithm', 'Action Space'])
uuid_list = {}
for alg, action in best_solutions[['Algorithm', 'Action Space']].drop_duplicates().itertuples(index=False):
    uuid_list[alg, action] = best_solutions[(best_solutions['Algorithm'] == alg) & (best_solutions['Action Space'] == action)]['Solution'].tolist()
with open(ROOT.joinpath('evaluation', 'best_solutions.pkl'), 'wb') as f:
    pickle.dump(uuid_list, f)

In [ ]:
result_df_norm_time = eval_utils.normalize_time(result_df_clean, group_by=['Task Set', 'Action Space', 'Algorithm', 'Task ID', 'Seed'], sampling_time='30s')

In [ ]:
result_df_norm_time['Task Set'].unique()

# Basic statistics

In [ ]:
eval_utils.print_step_count(result_df_clean)

# Convergence per task set

In [ ]:
local_df = result_df_norm_time.copy()  # Plot cost not reward
local_df['Maximum Reward'] = -1. * local_df['Maximum Reward']

In [ ]:
local_df['Action Space'] = local_df['Action Space'].replace({'xyz': 'Position', 
                                                             'xyz_rotvec': 'Position + Orientation'})
local_df['Task Set'] = local_df['Task Set'].replace({
    'test_simple': 'Simple', 
    'test_hard': 'Hard', 
    'test_realworld': 'Real',
    'test_edge_medium': 'Edge Medium',
    'test_edge_hard': 'Edge Hard'})

In [ ]:
with plt.style.context(['default', 'science', 'ieee', 'std-colors']):
    sns.set_style("darkgrid")
    f = sns.relplot(
        data=local_df, 
        x='Run Time', 
        y='Maximum Reward', 
        col='Task Set', 
        row='Action Space',
        # style='Action Space', 
        hue='Algorithm', 
        kind='line', 
        estimator='mean', 
        # err_style='bars',
        facet_kws={'sharey': False}, 
        hue_order=['Dummy', 'Random', 'BO', 'GA', 'AdamOptimizer'],
        col_order=['Simple', 'Hard', 'Real', 'Edge Medium', 'Edge Hard'],
        aspect=4 / 3, 
        height=7.16 / 4,  # two column is 7.16 inches wide
        seed=42)  # Fix for reproducible confidence intervals

In [ ]:
for idx, ax in enumerate(f.axes.flat):
    ax.set_xlabel(r"Run Time $[\si{\second}]$")
    if idx % 5 == 0:
        ax.set_ylabel(r"Minimum Cost $[\si{\second}]$")
    else:
        ax.set_ylabel(None)
    ax.set_xlim(pd.Timestamp(0, unit="s"), pd.Timestamp(timeout, unit="s"))
    ax.set_xticks(
        [pd.Timestamp(t, unit="s") for t in np.linspace(0, timeout, 5)], labels=[f"{t:.0f}" for t in np.linspace(0, timeout, 5)])
    ax.set_title(ax.get_title().replace('Task Set = ', ''))
    ax.set_title(ax.get_title().replace('Action Space = ', ''), fontsize=10)
sns.move_legend(f, "upper center", bbox_to_anchor=(0.48, 0), ncol=4)
f._legend.set_title(None)

In [ ]:
# Normalize y-axis per column
for col in range(f.axes.shape[1]):
    y_lim_lower = min(a.get_ylim()[0] for a in f.axes[:, col])
    y_lim_upper = max(a.get_ylim()[1] for a in f.axes[:, col])
    for row in range(f.axes.shape[0]):
        f.axes[row, col].set_ylim(y_lim_lower, y_lim_upper)

In [ ]:
plt.show()

In [ ]:
f.savefig(ROOT.joinpath('evaluation', 'ConvergencePlot.pdf'), bbox_inches='tight')

# Create Success Rate Table

In [ ]:
success_per_trial = result_df_clean.groupby(['Task Set', 'Action Space', 'Algorithm', 'Task ID', 'Seed'])['Success'].any()
success_per_trial *= 100  # Convert to percentage
tab_success_rate = success_per_trial.groupby(['Action Space', 'Task Set', 'Algorithm']).mean().unstack(-1).reindex(['test_simple', 'test_hard', 'test_realworld', 'test_edge_medium', 'test_edge_hard'], level=1)[['Dummy', 'Random', 'BO', 'GA', 'AdamOptimizer']]
tab_success_rate

In [ ]:
# Create table succ_rate_diff
table_string = tab_success_rate.to_latex(float_format='%.2f')
table_string = re.sub(r'([+-]?[0-9]*[.][0-9]+)', r'$\\qty{\g<1>}{\\percent}$', table_string)
table_string = re.sub(r'test_simple', 'Simple', table_string)
table_string = re.sub(r'test_hard', 'Hard', table_string)
table_string = re.sub(r'test_realworld', 'Real', table_string)
table_string = re.sub(r'test_edge_medium', 'Edge Medium', table_string)
table_string = re.sub(r'test_edge_hard', 'Edge Hard', table_string)
table_string = re.sub(r'xyz\}', r'Position}', table_string)
table_string = re.sub(r'xyz_rotvec', r'\\parbox\{1.2cm\}\{Position + Rotation}', table_string)
table_string = re.sub(r'\[t\]', r'', table_string)
print(table_string)

## Best cost in different action spaces

In [ ]:
result_per_trial = result_df_clean[result_df_clean['Task Set'] == 'test_simple'].groupby(['Action Space', 'Algorithm', 'Task ID', 'Seed'])['Maximum Reward'].max()  # Check for different task sets
result_per_trial = result_per_trial.reset_index()
result_per_trial['Hue'] = result_per_trial['Action Space'] + ' ' + result_per_trial['Algorithm']
# sns.kdeplot(result_per_trial, x='Maximum Reward', clip=(-20., 0.), hue='Hue')
# plt.show()

In [ ]:
result_per_trial.groupby(['Action Space', 'Algorithm'])['Maximum Reward'].describe()

In [ ]:
result_per_trial['Minimum Cost'] = -result_per_trial['Maximum Reward']
result_per_trial['Action Space'].replace({'xyz': 'Position', 
                                          'xyz_rotvec': 'Position + Rotation'},
                                           inplace=True)

In [ ]:
with plt.style.context(['default', 'science', 'ieee', 'std-colors']):
    sns.set_style("darkgrid")
    fig = plt.figure(figsize=(3.4, 2.55))  # IEEE column width and 4:3
    sns.boxplot(
        data=result_per_trial, 
        x='Algorithm', 
        y='Minimum Cost', 
        hue='Action Space', 
        order=['Dummy', 'Random', 'BO', 'GA', 'AdamOptimizer'],
        palette=sns.color_palette()[4:6],
        flierprops=dict(marker='*', markersize=2, linestyle='none', alpha=0.5),
        ax=plt.gca())
    
    fig.gca().set_ylabel(r"Minimum Cost $[\si{\second}]$")
    fig.gca().set_xlabel(None)
    fig.gca().legend(loc='upper center', title=None)
    sns.move_legend(fig.gca(), "upper center", bbox_to_anchor=(0.5, 1.12), ncol=2)

In [ ]:
plt.show()

In [ ]:
fig.savefig(ROOT.joinpath('evaluation', 'AlgDomainCostPlot.pdf'), bbox_inches='tight')

# Check if some tasks need rotvec

In [ ]:
# Create dataframe with final reward for each task, action space, algorithm, and seed
local_df = result_df_clean[result_df_clean['Algorithm'] != 'Dummy'].copy()
final_reward = local_df.groupby(['Action Space', 'Algorithm', 'Task ID', 'Seed'])['Maximum Reward'].last()
final_reward = final_reward.reset_index()
final_reward

In [ ]:
rotvec_final_reward = final_reward[final_reward['Action Space'] == 'xyz_rotvec']
rotvec_final_reward = rotvec_final_reward.set_index(['Task ID', 'Algorithm', 'Seed'])
rotvec_final_reward

In [ ]:
xyz_final_reward = final_reward[final_reward['Action Space'] == 'xyz']
xyz_final_reward = xyz_final_reward.set_index(['Task ID', 'Algorithm', 'Seed'])
xyz_final_reward

In [ ]:
# Order by difference in mean to find tasks that profit from rotvec
delta_mean = (final_reward[final_reward['Action Space'] == 'xyz'].groupby(['Task ID', ])['Maximum Reward'].mean() - 
              final_reward[final_reward['Action Space'] == 'xyz_rotvec'].groupby(['Task ID', ])['Maximum Reward'].mean())
order_mean = delta_mean.sort_values(ascending=False).index
delta_mean

In [ ]:
delta = (xyz_final_reward['Maximum Reward'] - rotvec_final_reward['Maximum Reward']).reset_index()  # TODO : This is point-wise compare; rather want to compare random draws over seeds and their expected difference
sns.barplot(data=delta, x='Task ID', y='Maximum Reward', order=order_mean[:10].append(order_mean[-10:]))
plt.xticks(rotation=90)
plt.show()

In [ ]:
# Try via permutation test
from scipy.stats import permutation_test

def statistic(x, y, axis):
    return np.mean(x, axis=axis) - np.mean(y, axis=axis)

def test_statistic(x, y, alternative):
    res = permutation_test((x, y), statistic, vectorized=True, n_resamples=100_000, alternative=alternative, random_state=42)
    print(f"{res.pvalue = }")
    print(f"{res.statistic = }")
    plt.figure()
    plt.hist(res.null_distribution)
    plt.show()

# Test high mean difference
best_task_id = order_mean[0]
print(f"{best_task_id = }")
test_statistic(
    xyz_final_reward.loc[best_task_id].reset_index()['Maximum Reward'],
    rotvec_final_reward.loc[best_task_id].reset_index()['Maximum Reward'],
    'greater')
# Test center mean difference
median_task_id = order_mean[len(order_mean) // 2]
print(f"{median_task_id = }")
test_statistic(
    xyz_final_reward.loc[median_task_id].reset_index()['Maximum Reward'],
    rotvec_final_reward.loc[median_task_id].reset_index()['Maximum Reward'],
    'two-sided')
# Test low mean difference
worst_task_id = order_mean[-1]
print(f"{worst_task_id = }")
test_statistic(
    xyz_final_reward.loc[worst_task_id].reset_index()['Maximum Reward'],
    rotvec_final_reward.loc[worst_task_id].reset_index()['Maximum Reward'],
    'less')

# Filter runtime statistics

In [ ]:
df = result_df_clean.copy()
df['Step Time'] = (df['Step Time'].copy() - pd.Timestamp(0, unit="s")).dt.total_seconds()
df['Fail Reason'] = df['Fail Reason'].replace(
    {'Path planning timeout': 'Path planning failed',
     'Invalid Path': 'Path planning failed'}
)
# Proper multi-line lables
label_order = {
    'Robot Length Filter': 'Robot\nLength',
    'Simple IK Filter': 'Simple\nIK',
    'IK Filter': 'IK\nFilter',
    'Path planning failed': 'Path\nplanning',
    'No failure': 'No\nfailure',
}
df['Fail Reason'] = df['Fail Reason'].replace(label_order)
fail_reasons = [f for f in eval_utils.FAIL_REASON_ORDER if f in df['Fail Reason'].unique()]

In [ ]:
assert len(df[df['Fail Reason'].isna()]['Valid Solution'].unique()) == 1, "There are NAs in Fail Reason without a valid solution"
df['Fail Reason'].fillna('No\nfailure', inplace=True)
df['Fail Reason'].unique()

In [ ]:
with plt.style.context(['default', 'science', 'ieee', 'std-colors']):
    sns.set_style("darkgrid")
    fig = plt.figure(figsize=(3.4, 2.55))  # IEEE column width and 4:3
    sns.boxplot(data=df, 
                x='Fail Reason',
                y='Step Time', 
                hue='Algorithm',  
                order=tuple(label_order.values()), 
                hue_order=['Dummy', 'Random', 'BO', 'GA', 'AdamOptimizer'],
                # whis=(.135, 99.865),  # 3 sigma
                flierprops=dict(marker='*', markersize=1, linestyle='none', alpha=0.3),
                ax=fig.gca(),
                )
    fig.gca().set_yscale('log')
    fig.gca().set_ylabel('Step Time $[\si{\second}]$')
    fig.gca().set_xlabel(None)
    fig.gca().legend(loc='lower right', title=None)
    # fig.gca().set_title('Step Time per Algorithm and Failure Reason')

In [ ]:
plt.show()

In [ ]:
fig.savefig(ROOT.joinpath('evaluation', 'runtime_per_filter.png'), bbox_inches='tight')

In [ ]:
result_df_clean.groupby(['Algorithm', 'Task Set'])['Step Time'].describe().reindex(['Dummy', 'Random', 'BO', 'GA', 'AdamOptimizer'])